## Add columns to csv file with contitional entries
read csv files from folder and add columns to each file with new entries based on the previous columns.



In [2]:
import os
import re
import csv

#folder_path = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\transcripts_csv_test"
#folder_path = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\transcripts_csv"
folder_path = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\test"


dataAll = {}

key_dict = {'.': 'final', 
            ',': 'continuing', 
            '?': 'rising', 
            '--': 'truncated'
            }

def ends_with_any_key(input_string, key_dictionary):
    """does the puntuation at the end match the keys in the dictionary"""
    for key in key_dictionary.keys():
        if input_string.endswith(key):
            return key_dictionary[key]
    return 'NA'
    
    
def wordCounter(wordList):
    """is it one word or more?"""
    #cleanedWordList = remove_non_alphanumeric(wordList)
            
    if len(wordList) == 1:
        return "Y"
    elif len(wordList) == 0:
        return '-'
    return "N"

def remove_spaces_before_punctuation(input_string):
    return re.sub(r'\s+([,.!?;:])', r'\1', input_string)



def filter_words(words_list):
    filtered_list = [word for word in words_list if all(c.isalnum() or c in '~=!.,:;[]' for c in word)]
    #print (filtered_list)
    return filtered_list

def remove_non_alphanumeric_strings(input_list):
    cleaned_list = [item for item in input_list if any(c.isalnum() for c in item)]
    return cleaned_list

def split_string_with_punctuation(input_string):
    pattern = r'\s+|\''
    """Use regex to split the string by words and punctuation"""
    words = re.split(pattern, input_string)
    #print (words)
    words = filter_words(words)
    words = remove_non_alphanumeric_strings(words)
    return words

def find_strings_with_symbols(string_list, symbol_list):
    # find all strings from list which don't contain any of the symbols in the other list
    filtered_strings = [s for s in string_list if not any(symbol in s for symbol in symbol_list)]
    return filtered_strings

# List all files in the folder
file_list = os.listdir(folder_path)

# Filter out only CSV files
csv_files = [file for file in file_list if file.endswith('.csv')]

# Process CSV files
for csv_file in csv_files:
    dataAll[csv_file] = []
    csv_file_path = os.path.join(folder_path, csv_file)
    new_csv_file_path = os.path.join(folder_path, f"clean\{csv_file}")
    
    dataAllFile = os.path.join(folder_path, f"clean\dataAll.csv")

    # Add headers and copy data to new CSV files
    with open(csv_file_path, 'r', newline='', encoding="utf8") as input_file, \
         open(new_csv_file_path, 'w', newline='',encoding="utf8") as output_file:

        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Read existing headers
        existing_headers = next(csv_reader)

        # Add new headers to existing headers
        new_headers = existing_headers + ["Intonation", "OneWordIU","IU_clean", "text"]
        csv_writer.writerow(new_headers)

        # Copy data from the old CSV to the new one
        for row in csv_reader:
            #print ('-------------------------')
            #print (row)
            
            # add Intonation column
            result = ends_with_any_key(row[5].strip(), key_dict)
            modified_row = row + [result]
            
            
            
            #add OneWordIU column
            
            # remove all comments between ( and )
            wordString = row[5].strip()
            
            
            
            if ("(" in wordString) and (")" in wordString):
                #print ('---------------------------')
                #print (wordString)
                res = re.findall(r'\(.*?\)', wordString)
                #print (wordString)
                for x in res:
                    wordString = wordString.replace(x, "")
                #print (wordString)
                
            
            # remove all <This  and that>
            if "<" in wordString:
                #print ('---------------------------')
                #print (wordString)
                
                res = re.findall(r"<\S*", wordString)
                
                for x in res:
                    wordString = wordString.replace(x, "")
                #print (wordString)
            
            if ">" in wordString:
                #print ('---------------------------')
                #print (wordString)
                res = re.findall(r"\S*>", wordString)
                
                for x in res:
                    wordString = wordString.replace(x, "")
                #print (wordString)
            
            
            #print (wordString)
            wordList = split_string_with_punctuation(wordString)
            #print (wordList)
            result2 = wordCounter(wordList)
            #print (result2)
            
            # override result for special case of word with symbols inside lile '[y]es'
            if (len(row[5].strip()) > 0 ) and (' ' not in row[5].strip()):
                result2 = 'Y'
                
            modified_row2 = modified_row + [result2]
            
            
            
            # clean the IU column and add a column with the clean text
            
            rawString = row[5].strip()
            
            rawString = rawString.replace('\t', '')
            rawString = rawString.replace('\n', '')
            rawString = rawString.replace('’', "'")
            
            # for devbugging
            if 'Xfrrr' in rawString:
                print ('---------------------------')
                printer = True
            else:
                printer = False
                
            if printer:
                print(rawString)
            
            pattern = r"\(\((.*?)\)\)"

            matches = re.findall(pattern, rawString)
            matchesComplete = [f"(({match}))" for match in matches] 
            for each in matchesComplete:
                rawString = rawString.replace(each, '')


            
            #remove '[Number' and 'Number]'
            patterns = [r"\[\d", r"\d\]"]
            for pattern in patterns:
                matches = re.findall(pattern, rawString)
                for match in matches:
                    rawString = rawString.replace(match, '')

            
            rawList = rawString.split()
            
            
            newList = []
            for each in rawList:
                pattern = r"!\w+[^A-Za-z0-9_]"

                matches = re.findall(pattern, each)
                if len(matches) > 0:
                    newList.append(matches[0][1:])
                else:
                    newList.append(each)
                
                    
            rawList = newList   
            
            
                
            
            # keep punctuation after > 
            newList = []
            for each in rawList:
                if '>' in each:
                    pattern = r">([^\w\s])"
                    matches = re.findall(pattern, each)
                    if len(matches) > 0:
                        newList[-1] = newList[-1] + matches[0]
                else:
                    newList.append(each)
            #print (newList)
            rawList = newList
                    
            
            
            # filter out all words from list which have one of the sybmols in it
            # symbols to be filtered out:
            symbols = [
                       "...",
                       "..",
                       "…(N.N)",
                       "!",
                       "X ",
                       "(",
                       ")",
                       "<",
                       ">",
                       "%"]
            
            
            filtered_strings = find_strings_with_symbols(rawList, symbols)
            
            outString = " ".join(filtered_strings)
            
            
            # replacement of some characters
            replaceDict = {"[": "", 
                           "]": "",
                           "=": "",
                           "~": "",
                           "@": "",
                           "--": "", 
                           "- ": " ",
                           " -": " "}
            
            for k, v in replaceDict.items():
                outString = outString.replace(k, v)
               

            if printer:
                print(outString)
                
            result3 = outString
            
            modified_row3 = modified_row2 + [result3]
            
            
            # version without the Xs
            result4 = outString.replace('X', '')
            result4 = remove_spaces_before_punctuation(result4)
            
            dataAll[csv_file].append(result4)
            
            
            if printer:
                print (result4)
            modified_row4 = modified_row3 + [result4]
            
            csv_writer.writerow(modified_row4)

    print(f"Processed CSV file: {csv_file}")
    
    

Processed CSV file: Bcnt_AEF_032_Camila.csv


In [2]:
# create a single file for all texts from the corpus
def filter_utf8_strings(strings):
    utf8_strings = []
    for string in strings:
        try:
            string.encode('utf-8').decode('utf-8')
            utf8_strings.append(string)
        except UnicodeDecodeError:
            pass
    return utf8_strings

def remove_multiple_spaces(input_string):
    return re.sub(r'\s+', ' ', input_string)

def write_dict_to_csv(dictionary, file_name):
    with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['text_name', 'text'])  # Write header

        for key, value in dictionary.items():
            
            value = filter_utf8_strings(value)
            
            cleanString = ""
            for each in value:
                
                
                
                
                if len(each) > 0: 
                    cleanString = cleanString + each.strip() + " "
                    
            
            cleanString = remove_multiple_spaces(cleanString)
                
               
                
            
            csv_writer.writerow([key, cleanString])
    #print (cleanString)
# Example dictionary

# Example usage
write_dict_to_csv(dataAll, dataAllFile)

print ("+++ Done +++")

+++ Done +++


In [12]:
def write_dict_to_files(dictionary):
    for key, value in dictionary.items():
        newName = key[:-3] + "txt"
        new_txt_file_path = os.path.join(folder_path, f"clean\\transcripts\{newName}")
        with open(new_txt_file_path, 'w', encoding="utf-8") as file:
            value = filter_utf8_strings(value)
            
            cleanString = ""
            for each in value:
                
                
                
                
                if len(each) > 0: 
                    cleanString = cleanString + each.strip() + " "
                    
            
            cleanString = remove_multiple_spaces(cleanString)
            
            file.write(cleanString)

write_dict_to_files(dataAll)
print ("+++ DONE +++")

+++ DONE +++
